#Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importing the packages

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import keras
import sklearn
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,plot_roc_curve,log_loss,f1_score,precision_score,recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
import seaborn as sn
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten, Dropout,LeakyReLU
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report,roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from keras.models import Sequential
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore')

%matplotlib inline


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### Importing the dataset

In [3]:
df = pd.read_csv('/content/drive/MyDrive/COVID19/Covid-19 Dataset_Clinical Data/ISRmergedtrain+test.csv')

#Split dataset 

In [4]:
X=df.values[:,0:8]
y=df.values[:,8]

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30)

#SMOTE

In [6]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1))) 
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0))) 
  

Before OverSampling, counts of label '1': 5861
Before OverSampling, counts of label '0': 63601 



In [7]:
# import SMOTE module from imblearn library 
# pip install imblearn (if you don't have imblearn in your system) 
from imblearn.over_sampling import SMOTE 
sm = SMOTE() 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel()) 
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape)) 
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0))) 

After OverSampling, the shape of train_X: (127202, 8)
After OverSampling, the shape of train_y: (127202,) 

After OverSampling, counts of label '1': 63601
After OverSampling, counts of label '0': 63601


#Metrics

In [8]:
def calculate_sensitivity_specificity(y_test, y_pred_test):
    # Note: More parameters are defined than necessary. 
    # This would allow return of other measures other than sensitivity and specificity
    
    # Get true/false for whether a breach actually occurred
    actual_pos = y_test == 1
    actual_neg = y_test == 0
    
    # Get true and false test (true test match actual, false tests differ from actual)
    true_pos = (y_pred_test == 1) & (actual_pos)
    false_pos = (y_pred_test == 1) & (actual_neg)
    true_neg = (y_pred_test == 0) & (actual_neg)
    false_neg = (y_pred_test == 0) & (actual_pos)
    
    # Calculate accuracy
    accuracy = np.mean(y_pred_test == y_test)
    
    # Calculate sensitivity and specificity
    sensitivity = np.sum(true_pos) / np.sum(actual_pos)
    specificity = np.sum(true_neg) / np.sum(actual_neg)
    
    return sensitivity, specificity, accuracy

#ADABOOST

In [ ]:
feat_cols = [2,3,4, 5,7]
abc1= AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=4,min_samples_leaf=1, min_samples_split=2 ), learning_rate=0.1, n_estimators=1000)
# Train Adaboost Classifer
abc1.fit(X_train_res[:,feat_cols], y_train_res)
y_predadb = abc1.predict(X_test[:,feat_cols])
# accuracy_score(y_test,y_predadb)
p=precision_score(y_test, y_predadb, average='micro')
r=recall_score(y_test, y_predadb,average='micro')
f=f1_score(y_test, y_predadb,average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predadb)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))



#MLP Classifier

In [10]:
feat_cols=[0,1, 2, 3, 4, 7]
classifier= MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=500,activation = 'relu',alpha=1e-5,solver='adam',random_state=16)
classifier.fit(X_train_res[:,feat_cols], y_train_res)
y_predmlp1 = classifier.predict(X_test[:,feat_cols])
accuracy_score(y_test,y_predmlp1)
p=precision_score(y_test, y_predmlp1 , average='micro')
r=recall_score(y_test, y_predmlp1 , average='micro')
f=f1_score(y_test, y_predmlp1 , average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predmlp1)

print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))



Accuracy:    92.70%
Sensitivity:   92.70%
Specificity:  93.82%
Precision:  92.70%
Recall:  92.70%
F1_score: 92.70%
AUC: 93.26%


#XGBoost Classifier

In [11]:
from xgboost import XGBClassifier

feat_cols=[1,3,4,5,7]
xgboost1 = XGBClassifier(max_depth=5,learning_rate=0.1,n_estimators=1000, subsample= 0.9,colsample_bytree=1,
gamma= 5)
xgboost1.fit(X_train_res[:,feat_cols], y_train_res)
# make predictions for test data
predxgboost1 = xgboost1.predict(X_test[:,feat_cols])
accuracy_score(y_test,predxgboost1)
p=precision_score(y_test, predxgboost1,average='micro')
r=recall_score(y_test, predxgboost1,average='micro')
f=f1_score(y_test, predxgboost1,average='micro')
sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, predxgboost1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))

Accuracy:    92.59%
Sensitivity:   92.59%
Specificity:  93.85%
Precision:  92.59%
Recall:  92.59%
F1_score: 92.59%
AUC: 93.22%


#SVM

In [12]:
from sklearn import svm

In [13]:
feat_cols=[0,1, 2, 4, 7]
SVM1 = svm.SVC(kernel='linear',C=0.1) # Linear Kernel
SVM1.fit(X_train_res[:, feat_cols], y_train_res)
#Predict the response for test dataset
y_predsvm1 = SVM1.predict(X_test[:,feat_cols])
p=precision_score(y_test, y_predsvm1,average='micro')
r=recall_score(y_test, y_predsvm1,average='micro')
f=f1_score(y_test, y_predsvm1,average='micro')
sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predsvm1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))

Accuracy:    92.59%
Sensitivity:   92.59%
Specificity:  93.85%
Precision:  92.59%
Recall:  92.59%
F1_score: 92.59%
AUC: 93.22%


#Random Forest Classifier



In [14]:
feat_cols=[1,2,3,4,7]

In [15]:

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

RFC1=RandomForestClassifier(n_estimators=500, max_depth=4)

#RFC1=RandomForestClassifier(n_estimators=2000,random_state=450)
#Train the model using the training sets y_pred=clf.predict(X_test)
RFC1.fit(X_train_res[:,feat_cols],y_train_res)
y_predrf1=RFC1.predict(X_test[:,feat_cols])
a=accuracy_score(y_test, y_predrf1 )
p=precision_score(y_test, y_predrf1 , average='weighted')
r=recall_score(y_test, y_predrf1 , average='weighted')
f=f1_score(y_test, y_predrf1, average='weighted')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predrf1)
print ("Accuracy:    %.2f%%"  %  (a*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))


Accuracy:    92.70%
Sensitivity:   92.70%
Specificity:  93.82%
Precision:  94.43%
Recall:  92.70%
F1_score: 93.31%
AUC: 93.26%


#Logistic Regression

In [16]:
feat_cols=[2,3,6]
lr1 = LogisticRegression()
lr1.fit(X_train_res[:,feat_cols],y_train_res)
y_predlr1=lr1.predict(X_test[:,feat_cols])
p=precision_score(y_test, y_predlr1 , average='micro')
r=recall_score(y_test, y_predlr1,average='micro')
f=f1_score(y_test, y_predlr1, average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_predlr1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))

Accuracy:    93.23%
Sensitivity:   93.23%
Specificity:  99.83%
Precision:  93.23%
Recall:  93.23%
F1_score: 93.23%
AUC: 96.53%



#Decision Tree classifier

In [17]:
feat_cols=[0,1, 2, 3, 4, 7]
dtc1 = DecisionTreeClassifier(max_depth=4)
dtc1.fit(X_train_res[:,feat_cols],y_train_res)
pred_dtc1=dtc1.predict(X_test[:,feat_cols])
p=precision_score(y_test, pred_dtc1 , average='micro')
r=recall_score(y_test, pred_dtc1 ,average='micro')
f=f1_score(y_test, pred_dtc1 , average='micro')

sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, pred_dtc1)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))

Accuracy:    86.40%
Sensitivity:   86.40%
Specificity:  86.52%
Precision:  86.40%
Recall:  86.40%
F1_score: 86.40%
AUC: 86.46%


# Gradient Boosting

In [18]:
feat_cols=[0,1, 2, 3, 5, 7]
GBC11=GradientBoostingClassifier(learning_rate=0.05,n_estimators=1000,max_depth=4)
GBC11.fit(X_train_res[:,feat_cols], y_train_res)
y_test_pred11 = GBC11.predict(X_test[:,feat_cols])
p=precision_score(y_test, y_test_pred11 , average='micro')
r=recall_score(y_test, y_test_pred11 , average='micro')
f=f1_score(y_test, y_test_pred11, average='micro')
sensitivity, specificity, accuracy = calculate_sensitivity_specificity(y_test, y_test_pred11)
print ("Accuracy:    %.2f%%"  %  (accuracy*100))
print ("Sensitivity:   %.2f%%"  %  (r*100))
print ("Specificity:  %.2f%%"  %  (specificity*100))
print ("Precision:  %.2f%%"  %  (p*100))
print ("Recall:  %.2f%%"  %  (r*100))
print ("F1_score: %.2f%%"  %  (f*100))
print ("AUC: %.2f%%"  %  (((specificity+r)/2)*100))


Accuracy:    91.71%
Sensitivity:   91.71%
Specificity:  92.85%
Precision:  91.71%
Recall:  91.71%
F1_score: 91.71%
AUC: 92.28%
